In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# Read all pk1 stargazer files from Data/st (sub)folders
pickled_dirs = []
repo_names = []
for folder in os.listdir('../Data/st'):
    try:
        pickled_dirs.append(f'../Data/st/{folder}/'+os.listdir('../Data/st/'+folder)[0])
        repo_names.append(folder)
    except:
        print('NO STARGAZER DATA IN: ', folder)
        continue

print('\n', pickled_dirs[:20])
print('\n', repo_names[:20])

NO STARGAZER DATA IN:  prisma-graphql-import

 ['../Data/st/AE9RB-ruby-redis/AE9RB_ruby-redis_st.pk1', '../Data/st/AigeStudio-Android/AigeStudio_Android_st.pk1', '../Data/st/AlburIvan-SlickForm/AlburIvan_SlickForm_st.pk1', '../Data/st/AlexLiuSheng-CheckVersionLib/AlexLiuSheng_CheckVersionLib_st.pk1', '../Data/st/AllenDowney-ThinkJavaCode/AllenDowney_ThinkJavaCode_st.pk1', '../Data/st/Aufree-ESTMusicIndicator/Aufree_ESTMusicIndicator_st.pk1', '../Data/st/Azure-azure-storage-fuse/Azure_azure-storage-fuse_st.pk1', '../Data/st/BMIRDS-deepslide/BMIRDS_deepslide_st.pk1', '../Data/st/BetterCloud-vault-java-driver/BetterCloud_vault-java-driver_st.pk1', '../Data/st/BreakingMalwareResearch-eleven/BreakingMalwareResearch_eleven_st.pk1', '../Data/st/BurntSushi-quickcheck/BurntSushi_quickcheck_st.pk1', '../Data/st/BycorSanchez-resources/BycorSanchez_resources_st.pk1', '../Data/st/CMU-Perceptual-Computing-Lab-openpose/CMU-Perceptual-Computing-Lab_openpose_st.pk1', '../Data/st/CVCalendar-CVCalendar/C

In [3]:
# Read in all pickled files
df_list = [pd.read_pickle(file_dir) for file_dir in pickled_dirs]

In [4]:
df_list[6][:5]

,node,starredAt
0,"{'createdAt': '2016-09-22T05:24:04Z', 'updated...",2017-09-12T18:38:08Z
1,"{'createdAt': '2016-05-23T16:22:55Z', 'updated...",2017-09-28T16:00:36Z
2,"{'createdAt': '2015-01-29T18:47:54Z', 'updated...",2017-09-29T19:47:09Z
3,"{'createdAt': '2010-12-29T14:25:26Z', 'updated...",2017-10-30T09:27:25Z
4,"{'createdAt': '2012-06-21T10:24:44Z', 'updated...",2017-11-02T02:53:31Z


In [5]:
def clean_stars_df(star_df: pd.DataFrame) -> pd.DataFrame:
    """Takes a DataFrame of issue data and returns a cleaned DataFrame."""

    node_dict_list = star_df['node'].values.tolist()
    unpacked_dict_list = []

    for item in node_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'createdAt': np.NaN, 'updatedAt': np.NaN, 'id': np.NaN, 'login': np.NaN, 'company': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    # Recombine issue_df and unpacked dicts
    node_star_df = pd.DataFrame(unpacked_dict_list)
    clean_star_df = star_df.drop(labels=['node'], axis='columns')
    clean_star_df['createdAt'] = node_star_df['createdAt']
    clean_star_df['updatedAt'] = node_star_df['updatedAt']
    clean_star_df['id'] = node_star_df['id']
    clean_star_df['login'] = node_star_df['login']
    clean_star_df['company'] = node_star_df['company']

    
    # Type conversion
    try:
        clean_star_df['createdAt'] = clean_star_df['createdAt'].apply(pd.Timestamp)
        clean_star_df['updatedAt'] = clean_star_df['updatedAt'].apply(pd.Timestamp)
    except:
        pass
    
    return clean_star_df

In [6]:
# Clean each star DataFrame
cleaned_stars = []
for df in df_list:
    cleaned_stars.append(clean_stars_df(df))

In [7]:
df_list[3][:5]

,node,starredAt
0,"{'createdAt': '2015-12-09T06:46:27Z', 'updated...",2016-09-26T09:15:06Z
1,"{'createdAt': '2013-01-31T18:07:47Z', 'updated...",2016-09-26T10:42:51Z
2,"{'createdAt': '2012-07-28T04:08:05Z', 'updated...",2016-09-26T12:09:11Z
3,"{'createdAt': '2012-06-06T03:26:48Z', 'updated...",2016-09-26T12:09:53Z
4,"{'createdAt': '2009-01-12T06:23:11Z', 'updated...",2016-09-26T12:51:36Z


In [8]:
cleaned_stars[3][:5]

,starredAt,createdAt,updatedAt,id,login,company
0,2016-09-26T09:15:06Z,2015-12-09 06:46:27+00:00,2019-06-27 02:37:41+00:00,MDQ6VXNlcjE2MjE5NDk5,talentjiang,None
1,2016-09-26T10:42:51Z,2013-01-31 18:07:47+00:00,2019-04-02 08:29:03+00:00,MDQ6VXNlcjM0NDA5NDU=,tanweijiu,AltarLamp
2,2016-09-26T12:09:11Z,2012-07-28 04:08:05+00:00,2019-08-23 13:12:31+00:00,MDQ6VXNlcjIwNTUyNDA=,dstmath,None
3,2016-09-26T12:09:53Z,2012-06-06 03:26:48+00:00,2019-08-07 14:59:39+00:00,MDQ6VXNlcjE4MjE1ODM=,draco1023,None
4,2016-09-26T12:51:36Z,2009-01-12 06:23:11+00:00,2019-07-11 02:42:33+00:00,MDQ6VXNlcjQ1ODk5,fengxueren,None


In [9]:
cleaned_dir = '../Data/cleaned_stars/'
for ind, df in enumerate(cleaned_stars):
    df.to_pickle(cleaned_dir+repo_names[ind]+'_st.pk1')

In [10]:
# Test pickled files
pd.read_pickle(cleaned_dir+repo_names[3]+'_st.pk1')[:5]

,starredAt,createdAt,updatedAt,id,login,company
0,2016-09-26T09:15:06Z,2015-12-09 06:46:27+00:00,2019-06-27 02:37:41+00:00,MDQ6VXNlcjE2MjE5NDk5,talentjiang,None
1,2016-09-26T10:42:51Z,2013-01-31 18:07:47+00:00,2019-04-02 08:29:03+00:00,MDQ6VXNlcjM0NDA5NDU=,tanweijiu,AltarLamp
2,2016-09-26T12:09:11Z,2012-07-28 04:08:05+00:00,2019-08-23 13:12:31+00:00,MDQ6VXNlcjIwNTUyNDA=,dstmath,None
3,2016-09-26T12:09:53Z,2012-06-06 03:26:48+00:00,2019-08-07 14:59:39+00:00,MDQ6VXNlcjE4MjE1ODM=,draco1023,None
4,2016-09-26T12:51:36Z,2009-01-12 06:23:11+00:00,2019-07-11 02:42:33+00:00,MDQ6VXNlcjQ1ODk5,fengxueren,None
